In [0]:
#===============================================================================
#                          IDS 561 Assignment 2                                 | 
#===============================================================================

In [0]:
#===============================================================================
#      WARNING - Run this only if you want to clear all the variable values     | 
#===============================================================================
%reset -f

In [0]:
#===============================================================================
#                   Importing all the required libraries                        |
#===============================================================================

import   pandas                        as pd
import   numpy                         as np

import nltk
from   nltk.tokenize                   import WhitespaceTokenizer
from   nltk.stem                       import PorterStemmer 
from   nltk.tokenize                   import word_tokenize 
from   nltk.corpus                     import stopwords
nltk.download('stopwords')

import string

from   sklearn.feature_extraction.text import TfidfTransformer
from   sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#===============================================================================
#                         Read the Training Data                                |
#===============================================================================

#Reading the input CSV file into a DataFrame
file_train = pd.read_csv('train_data.csv',encoding = "ISO-8859-1")

#Picking the required columns i.e. Sentiment and Tweet
c = [1,3]
file_train = file_train.iloc[:,c]
file_train.rename(columns={'SentimentText':'Tweet'}, inplace=True)
#file_train = file_train.iloc[:,:]


#===============================================================================
#                         Read the Test Data                                    |
#===============================================================================

#Reading the input CSV file into a DataFrame
file_test = pd.read_csv('train_data.csv',encoding = "ISO-8859-1")

#Picking the required columns i.e. Sentiment and Tweet
c = [1,3]
file_test = file_test.iloc[:,c]
file_test.rename(columns={'SentimentText':'Tweet'}, inplace=True)

#file_test = file_test.iloc[1:50,:]

In [0]:
#===============================================================================
# Train Data Cleaning- Removing Punctuations, Stop words and making Lowercase   |
#===============================================================================

train_file = file_train

#Removing Punctuations from the Tweet column in dataframe
train_file['Tweet'] = train_file['Tweet'].str.replace('[^\w\s]','')
#Converting all the words in Tweet column to lowercase
train_file['Tweet'] = train_file['Tweet'].str.lower()

#Removing all the numbers from the strings in dataFrame
train_file['Tweet'] = train_file['Tweet'].str.replace('\d+', '')

#Removing all the stopwords from the Tweet column in dataframe
from nltk.corpus import stopwords
stop = stopwords.words('english')
train_file['Tweet'] = train_file['Tweet'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

#===============================================================================
# Test Data Cleaning- Removing Punctuations, Stop words and making Lowercase    |
#===============================================================================

test_file = file_test

#Removing Punctuations from the Tweet column in dataframe
test_file['Tweet'] = test_file['Tweet'].str.replace('[^\w\s]','')
#Converting all the words in Tweet column to lowercase
test_file['Tweet'] = test_file['Tweet'].str.lower()

#Removing all the numbers from the strings in dataFrame
test_file['Tweet'] = test_file['Tweet'].str.replace('\d+', '')

#Removing all the stopwords from the Tweet column in dataframe
from nltk.corpus import stopwords
stop = stopwords.words('english')
test_file['Tweet'] = test_file['Tweet'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))


In [0]:
#===============================================================================
#                         TF-IDF for Train Data                                 |
#===============================================================================

train_tfidf = train_file

from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=5000)

#Vectorizer to get TF-IDF of the Tweet column
train_tfidf = v.fit_transform(train_tfidf['Tweet'])

#Convert the above TF-IDF vectorizer to a Pandas Dataframe
train_tfidf = pd.DataFrame(data = train_tfidf.toarray(), columns=v.get_feature_names())

#Fetching the sentiment column from the original dataframe
Tweet_train = train_file.iloc[:,0]

#Correcting the index of the column with index starting from 0
Tweet_train.index       = np.arange(0, len(Tweet_train)) 
train_tfidf.index = np.arange(0, len(train_tfidf))

#Now insert the sentiment column in the first column to the above TF-IDF Dataframe
train_tfidf.insert(0, "Tweet", Tweet_train)

#===============================================================================
#                         TF-IDF for Test Data                                  |
#===============================================================================

test_tfidf = test_file
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=5000)

#Vectorizer to get TF-IDF of the Tweet column
test_tfidf = v.fit_transform(test_tfidf['Tweet'])

#Convert the above TF-IDF vectorizer to a Pandas Dataframe
test_tfidf = pd.DataFrame(data = test_tfidf.toarray(), columns=v.get_feature_names())

#Fetching the sentiment column from the original dataframe
Tweet_test = test_file.iloc[:,0]

#Correcting the index of the column with index starting from 0
Tweet_test.index = np.arange(0, len(Tweet_test)) 

#Now insert the sentiment column in the first column to the above TF-IDF Dataframe
test_tfidf.insert(0, "Tweet", Tweet_test)

#train_tfidf - Contains the Train data with Tweet column as Target feature
#test_tfidf  - Contains the Test data  with Tweet column as Target feature

In [0]:
#Input Data to the Logistic Regression Model
X_train = train_tfidf
X_TRAIN = X_train.iloc[:,1:]

Y_train = train_tfidf.iloc[:,0]
Y_TRAIN = Y_train.to_frame()

X_TEST  = test_tfidf.iloc[:,1:]
Y_TEST  = test_tfidf.iloc[:,0]

In [0]:
#===============================================================================
#                         START OF LOGISTIC REGRESSION                          |
#===============================================================================

#STEP 1: Splitting the data into 10 Folds

#STEP 2: For each fold, based on 100 iterations we find the theta values
#        after using Gradient Descent          

#STEP 3: After finding the theta values for each fold, based on those values
#        we predict on the Validation data

#STEP 4: We find the Accuracy for that fold using those theta values

#STEP 5: We repeat this process for all 10 folds and find the BEST accuracy

#STEP 6: Whichever fold has the best accuracy, we pick the corresponding
#        theta values as the final theta values 
    
#STEP 7: The final theta values will be used for the final predictions
#===============================================================================

#K-Fold Cross Validation
from numpy import array
from sklearn.model_selection import KFold

#Prepare cross validation
kfold = KFold(10, True, 1)
bestaccuracy = 0
theta_final = np.zeros(X_TRAIN.shape[1])

# Enumerate splits
for train, test in kfold.split(X_TRAIN):
    X_train    = X_TRAIN.iloc[train,:]
    X_validate = X_TRAIN.iloc[test,:]
    
    Y_train    = Y_TRAIN.iloc[train,:]
    Y_validate = Y_TRAIN.iloc[test,:]
    
    X = X_train
    y = Y_train

    num_iter = 5

    for i in range(num_iter):
  
      #PART 1 : Sigmoid Function
      #Input Parameters - X and theta

      #Create a Zero values matrix whose dimensions are equal to the number of columns in Dataframe

      if 'theta_new' not in vars():  #Only if theta_new is blank fill theta values with 0's
        theta = np.zeros(X.shape[1]) #The feature coefficient matrix
  
      z = np.dot(X, theta)           #Z = X.θ Matrix Multiplication (i.e. Row is multipled by Columns)
      h = 1 / (1 + np.exp(-z))       #Appling Sigmoid function on X
      h = h.reshape(len(X),1)

      #PART 2 : Find the Gradient of Loss Function

      gradient = np.dot(X.T, (h - y)) / y.shape[0]

      learning_rate = 0.1
      weight        = theta
      theta_new     = weight - learning_rate * gradient
  
      theta_new     = pd.DataFrame(theta_new)
      theta_new     = theta_new.iloc[:,0].to_frame()
      theta         = theta_new 
    
    del theta_new #Clearing the variable after each fold

    #PART 3 : Making the Predictions on the Validation Data

    z    = np.dot(X_validate, theta) #Applying sigmoid on the new Theta values
    pred = 1 / (1 + np.exp(-z))
    
    #PART 4 : Calculating the Accuracy based on the actual target values

    predicted_class = ((pred >= 0.5) .astype(int))
    predicted_class = predicted_class.flatten()
    predicted_class = predicted_class.reshape(len(predicted_class),1)
    acc             = np.mean(predicted_class == Y_validate)
    
    #PART 5 : Finding the final theta value for the best accuracy obtained fold
     
    if(acc[0] > bestaccuracy):
        theta_final  = theta
        bestaccuracy = acc[0]

print('Best Accuracy: {0:0.2f}'.format(bestaccuracy))

Best Accuracy: 0.71


In [0]:
#Final Predictions based on the final theta values obtained

z    = np.dot(X_TEST, theta_final) #Applying sigmoid on the new Theta values
pred = 1 / (1 + np.exp(-z))

predicted_class = ((pred >= 0.5) .astype(int))
predicted_class = predicted_class.flatten()    #Like unlist in R


In [0]:
#Calculating the Recall, Precision and Average Precision-recall score

from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

average_precision = average_precision_score(Y_TEST, predicted_class)

prec = precision_score(Y_TEST, predicted_class)
recall = recall_score(Y_TEST, predicted_class)
conf = confusion_matrix(Y_TEST, predicted_class)

print('Average precision-recall score: {0:0.2f}'.format(average_precision))

print('Precision score: {0:0.2f}'.format(prec))

print('Recall score: {0:0.2f}'.format(recall))

Average precision-recall score: 0.68
Precision score: 0.70
Recall score: 0.86
